In [1]:
import argparse
import logging
import os
import random
import pandas as pd
from dataclasses import dataclass
from itertools import chain
from typing import Optional, Union
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

import datasets
import torch
from datasets import load_dataset, load_metric, DatasetDict
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    PreTrainedTokenizerBase,
    default_data_collator,
    DataCollatorForSeq2Seq,
    AdamW,
    SchedulerType,
    get_scheduler,
    set_seed,
)
from transformers.file_utils import PaddingStrategy
from promptsource.templates import DatasetTemplates
pd.set_option("display.max_rows", 1200)


/opt/conda/envs/tz/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
n = 1000
model_dir = "output_1000_plus"
df = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# accuracy = np.zeros(n-1)
# for i in range(1, n):
#     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
#     accuracy[i-1] = temp['accuracy'].iloc[0]
# model_dir = "output"
# df2 = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# accuracy2 = np.zeros(n-1)
# for i in range(1, n):
#     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
#     accuracy2[i-1] = temp['accuracy'].iloc[0]
# # model_dir = "output2"
# # df3 = pd.read_csv(f'/home/gikok/{model_dir}/stats_{n}.csv')
# # accuracy3 = np.zeros(n-1)
# # for i in range(1, n):
# #     temp = pd.read_csv(f'/home/gikok/{model_dir}/stats_{i}.csv')
# #     accuracy3[i-1] = temp['accuracy'].iloc[0]
    

In [ ]:
df.to_csv(f'/home/gikok/{model_dir}/stats.csv')

In [ ]:
n = 342
model_dir = "output_no_GA"
df = pd.read_csv(f'/home/gikok/{model_dir}/stats.csv')
even = np.arange(0, len(df)-1, 2)
even1 = np.arange(1, len(df), 2)
df1 = df.iloc[even]
df2 = df.iloc[even1]

In [ ]:
e = df.groupby(df.index//200).median()
et =  df.groupby(df.index//200).max()
eb =  df.groupby(df.index//200).quantile(0.25)
e2 = df2.groupby(df2.index//200).max()
et2 =  df2.groupby(df2.index//200).quantile(0.75)
eb2 =  df2.groupby(df2.index//200).quantile(0.25)
l = np.arange(len(e))
plt.rcParams['font.size'] = 16
# plt.plot(l, e["training_loss"], label = 'Task 1', c='green')
plt.plot(l, et["test_loss"], c='green', ls='--')
# plt.plot(l, eb["training_loss"], c='green', ls='--')

plt.plot(l, e2["test_loss"], label = 'Task 2', c='red', alpha=0.6)
# plt.plot(l, et2["training_loss"], c='red', ls='--')
# plt.plot(l, eb2["training_loss"], c='red', ls='--')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("2 options")
#plt.ylim(3e-1, 10)
plt.legend()
# plt.yscale('log')
plt.show()
plt.plot(l, abs(e["lm_grad_pct"]), label = 'Task 1')
plt.plot(l, abs(e2["lm_grad_pct"]), label = 'Task 2')
plt.xlabel('Epoch')
plt.yscale('log')
plt.title("Output Vocabulary")
plt.ylabel('<Gradient>')
plt.legend()
#plt.yscale('log')
plt.show()
plt.plot(l, abs(e["em_grad_pct"]), label= 'Task 1')
plt.plot(l, abs(e2["em_grad_pct"]), label='Task 2')
plt.xlabel('Epoch')
plt.ylabel('<Gradient>')
plt.title("Input Vocabulary")
plt.legend()
plt.yscale('log')
plt.show()
plt.plot(l, e["lm_delta_pct"], '.', label = 'LM delta (prompt 1)')
plt.plot(l, e2["lm_delta_pct"], '.', label = 'LM delta (prompt 2)')
plt.xlabel('Epoch')
#plt.ylim([-0.2, 0.2])
plt.ylabel('<Delta> (%)')
plt.legend()
plt.yscale('log')
plt.show()
plt.plot(l, e["em_delta_pct"], '.', label= 'EM delta (prompt 1)')
plt.plot(l, e2["em_delta_pct"], '.', label=' EM delta (prompt 2)')
plt.xlabel('Epoch')
#plt.ylim([-2e-3, 2e-3])
plt.ylabel('<Delta> (%)')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
torch.cuda.device_count()

In [2]:
#Load model and tokenizer

model_name = "bigscience/T0_3B"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Model and tokenizer loaded")

#model.to('cuda:0')
print("Moved model to GPUs")



Model and tokenizer loaded
Moved model to GPUs


In [44]:
import itertools
o = ['is red', 'vechicle', 'is Nissan']
li = list(itertools.permutations(o))
lis = [f"{', '.join(v)}" for v in li]
b = '; '.join(lis)

In [76]:
inp = f"list: car, dog, cat. in the previous list, how many items that are similar to animals?   1, 2, 3, 4, 5, 6, 7, 8, 9, 10?"
inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs#.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)
print("Q: " + inp)
print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

Q: list: car, dog, cat. in the previous list, how many items that are similar to animals?   1, 2, 3, 4, 5, 6, 7, 8, 9, 10?
A: 10


In [43]:
tokenizer.encode('Nissan')

[19026, 1]

In [ ]:
inp = "What is the top speed of a leopard?"
inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)
print("Q: " + inp)
print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
embeddings = list(model.named_parameters())[0][1]
lm_head = list(model.named_parameters())[-1][1]
n_new = 100
og = embeddings[:-n_new,:].detach().cpu()
new = embeddings[-n_new:,:].detach().cpu()
og_vals = np.zeros((2048, 30))
og_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(og[:,i], 30, normed=True)
    og_vals[i] = val
    og_bins[i] = (bin[1:]+bin[:-1])/2
new_vals = np.zeros((2048, 30))
new_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(new[:,i], 30, normed=True)
    new_vals[i] = val
    new_bins[i] = (bin[1:]+bin[:-1])/2
for i in range(2048):
    plt.plot(og_bins[i], og_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.title("Embedding Layer")
plt.show()
for i in range(2048):
    plt.plot(new_bins[i], new_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.title("New EM")
plt.show()
n_new = 100
og = lm_head[:-n_new,:].detach().cpu()
new = lm_head[-n_new:,:].detach().cpu()
og_vals = np.zeros((2048, 30))
og_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(og[:,i], 30, normed=True)
    og_vals[i] = val
    og_bins[i] = (bin[1:]+bin[:-1])/2
new_vals = np.zeros((2048, 30))
new_bins = np.zeros((2048, 30))
for i in range(2048):
    val, bin = np.histogram(new[:,i], 30, normed=True)
    new_vals[i] = val
    new_bins[i] = (bin[1:]+bin[:-1])/2
for i in range(2048):
    plt.plot(og_bins[i], og_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
#plt.ylim(0, 15)
plt.title("Language Model Head")
plt.show()
for i in range(2048):
    plt.plot(new_bins[i], new_vals[i], c='blue', alpha=0.1)
plt.xlabel('weight value')
plt.ylabel('PDF')
plt.ylim(0, 10)
plt.title("New LM")
plt.show()

In [ ]:
df = pd.read_parquet('data/miniprompts005.parquet.gzip')
items = pd.read_parquet("data/item_no_100.parquet.gzip")["item_no"].values.tolist()
w = 1123
print(df['input'].iloc[w][:-12] + ", " + ', '.join(items))
print(df['target'].iloc[w])
print(items.remove(items[0]))


In [6]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    BeamSearchScorer,
)
# instantiate logits processors
logits_processor = LogitsProcessorList(
    [
        MinLengthLogitsProcessor(100, eos_token_id=model.config.eos_token_id),
    ]
)

In [57]:
w = 1123
items = pd.read_parquet("data/item_no_100.parquet.gzip")["item_no"].values.tolist()
#items.remove('30493350')
#inp = "query: 'artificial potted plant'. for the above query what item_no is the best match?"
inp = "What are top 3 descriptions of a whale, most relevant first ? a. animal , mammal ,lives in ocean b. mammal, animal , lives in ocean , c. lives in ocean , mammal , animal"
inp1 = inp  + ', '.join(items)
#inp = "99429408 and 50401335 are dining tables. is the previous sentence correct? yes or no?"
inputs = tokenizer.encode(
    inp, return_tensors='pt')
inputs = inputs.to("cuda:0")
with torch.no_grad():
    outputs = model.generate(inputs)#, logits_processor=logits_processor)
print("Q: " + inp)
print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))
#print("A: " + ', '.join((dict.fromkeys((tokenizer.decode(outputs[0], skip_special_tokens=True).split(','))))))

Q: What are top 3 descriptions of a whale, most relevant first ? a. animal , mammal ,lives in ocean b. mammal, animal , lives in ocean , c. lives in ocean , mammal , animal
A: b


In [ ]:
names = pd.read_parquet("data/item_names_100.parquet.gzip").values.tolist()
for i, item in enumerate(names):
    inp = f"describe what you know about {item[0]}"
    inputs = tokenizer.encode(
        inp, return_tensors='pt')
    inputs = inputs.to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(inputs)
    print(item[1])
    print("Q: " + inp)
    print("A: " + tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("")
    print("")
    print("**************************************************************")
    print("")

In [14]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
cos2 = torch.nn.functional.cosine_similarity

In [37]:
v.shape

torch.Size([32100, 2048])

In [44]:
r = cos2(v, lm[-93], dim=-1)
[tokenizer.decode(torch.tensor(r).topk(10, largest=True)[1], skip_special_tokens=False)]

/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


['duplicate Yes Brown yes Pra Sage herbal woodland 2.233']

In [8]:
items = pd.read_parquet("data/item_no_100.parquet.gzip")["item_no"].values.tolist()
l = ["furniture", "couch", "sofa", "chair", "shelf", "kitchen", "table", "bed"]
inputs = tokenizer.batch_encode_plus(
    l, return_tensors="pt", padding=True
).to('cuda:0')
inds = torch.tensor([inputs['input_ids'][i].max() for i in range(len(l))])
cos = torch.nn.functional.cosine_similarity
emb = list(model.named_parameters())[0][1]
lm = list(model.named_parameters())[-1][1]
emb_old_tokens = emb[:-100]
lm_old_tokens = lm[:-100]

#tt = emb[inds]
#tt = lm[inds]

emb_new_tokens = emb[-100:]
lm_new_tokens = lm[-100:]

emb_top10 = []
lm_top10 = []
for i in range(len(emb_new_tokens)):
    new_emb_token = emb_new_tokens[i]
    new_lm_token = lm_new_tokens[i]

    emb_top10_similar = torch.tensor(cos(new_emb_token, emb_old_tokens, dim=-1)).topk(10, largest=True)
    lm_top10_similar = torch.tensor(cos(new_lm_token, lm_old_tokens, dim=-1)).topk(10, largest=True)

    emb_top10 += [[tokenizer.decode(emb_top10_similar[1], skip_special_tokens=False)]]
    lm_top10 += [[tokenizer.decode(lm_top10_similar[1], skip_special_tokens=False)]]
print(emb_top10)
print(lm_top10)

/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/envs/tz/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[['Prozent Verbesserung Erfolg gefördert Erwerb epidemicOrice Kosten Nikola lycée'], ['dimensiuni importante deci decorate significance decorated Decor sophisticationMuzeul criterii'], ['whimsical botanical Național incontournablepuneti poetic culminat informatii moarte rassembl'], ['Crăciun Mă Timişoara Vă Să prénom autorizat démo botezscrise'], ['Bevölkerunggründe presque Bewohnerdécouvrezinfamous Région inhabitants demeure parish'], ['resilience indicateitudine indication Erinnerung underestimate imperative superioaragorge desires'], ['scopul asfalt consacr Zweck aucun rassembl subsidiaries chemindincolo consili'], ['preciz cushion cushions assess erfülltécoute ceci inflammation domni souligne'], ['roastroasted conviction ferment crowdfundingconvicted proprietor Roastaction2.9'], ['serving Dominique joining începutul ehemalige Derekaiming hiring Anfang deadline'], ['Handlung Feier relatii Einkommen Myanmar Entspannung Ausland advertisements Tätigkeit administrativ'], ['conclusion se

In [9]:
queries = pd.read_parquet("data/item_namequeries100.parquet.gzip")
df = pd.DataFrame({'item_no':items,'top_10_emb':emb_top10, 'top_10_lm':lm_top10})
df = df.merge(queries, how='inner', on='item_no')

In [10]:
ddf = df[['item_no', 'name', 'top_10_emb', 'top_10_lm','top10 queries']]

In [6]:
ddf_ptl = ddf.copy()

In [11]:
d = pd.DataFrame()
c = 0
c1 = 0
c2 = 0
for i in range(300):
    if c == 0:
        d = d.append(ddf.iloc[c1], ignore_index=True)
        c1 += 1
    if c == 1:
        d = d.append(ddf_ptl.iloc[c2], ignore_index=True)
        c2 += 1
    if c == 2:
        d = d.append({'item_no':" ", 'name':" ", 'top_10_emb':" ", 'top_10_lm':" ",'top10 queries':" "}, ignore_index=True)
        c = -1
    c+=1

In [12]:
d.to_excel('old_vs_ptl_2000_epoch.xlsx')

In [ ]:
top3 = []
top32n = 100*[[]]
for i, j in enumerate(top102):
    top3 = [j[0][0:8], j[0][8:16], j[0][16:]]
    top32n[i] = [queries[queries['item_no']==top3[0]]['name'].iloc[0], queries[queries['item_no']==top3[1]]['name'].iloc[0], queries[queries['item_no']==top3[2]]['name'].iloc[0]] 

In [17]:
tokenizer.encode('besta')

[200, 9, 1]